# Homework 4:

## 1. Play with fasta files.

Create a program to analyse all the yeast sequences.

__FIRST CELL OF PROBLEM 1:__

a. Download all the coding sequences from yeast like in class.

In [62]:
import myFunctions as mf
import urllib.request as urreq
import gzip


url = "http://sgd-archive.yeastgenome.org/sequence/S288C_reference/orf_dna/orf_coding.fasta.gz"
try:
    urreq.urlretrieve(url,'./Resources/orf_coding.fasta.gz')  #first get the webpage you want
except:
    print("Something went wrong!")
else:
    print("Download worked!")

nucFile = gzip.open('./Resources/orf_coding.fasta.gz', 'rb') # rb is read as bianary file
file_content = nucFile.read() # get all of the contnets

outfile = './Resources/orf_coding.fasta' # put all of the unzipped contents
fileout = open(outfile, 'wb')
fileout.write(file_content)
fileout.close()

Download worked!


__SECOND CELL OF PROBLEM 1:__

b. Use the functions in `myFunctions.py` to read all the sequences in the file.

c. Calculate GC content and length of each sequence.

d. Make a dictionary of dictionaries with the sequence name as the key, and GC content, length, the sequence of each gene and the protein sequence derived from translating the gene.

e. Write the results to a tab delimited file, one sequence per line. Each line should contain, the name of the sequence, GC content, length, DNA seq, and Protein Seq, each of these fields should be separated by tabs.

In [64]:
# b) Read in the sequences
nucFile  = './Resources/orf_coding.fasta'
protFile = './Resources/orf_trans.fasta'

nucseqs, aaseqs = mf.readFasta(nucFile)

mf.writeFasta(protFile, aaseqs)

# c) find the GC content and Length
def gc_length(seq):
    gc_count = sum(1 for base in seq if base in "GCgc")
    gc_content = (gc_count / len(seq)) * 100
    sequence_length = len(seq)
    return gc_content, sequence_length

# d) create dictionary within dictionary - gc_content, length, nucleotide_sequence, protein_sequence
def gene_dict(nucseqs, aaseqs):
    info = {}
    for (nuc_name, nuc_seq), (_, aa_seq) in zip(nucseqs, aaseqs):
        gc_content, seq_length = gc_length(nuc_seq)

        info[nuc_name] = {
            'GC_content': gc_content,
            'length': seq_length,
            'nucleotide_sequence': nuc_seq,
            'protein_sequence': aa_seq
        }
    return info

# create dict using yeast files
gene_info_dict = gene_dict(nucseqs, aaseqs)

out_file = './Resources/gene_info.tsv'

# e) write a tsv file with the dictionary information
with open(out_file, 'w') as file:    
    for gene_name, gene_info in gene_info_dict.items():
        line = f"{gene_name}\t{gene_info['GC_content']}\t{gene_info['length']}\t{gene_info['nucleotide_sequence']}\t{gene_info['protein_sequence']}\n"
        file.write(line)

print(f"Data written to {out_file}")


Data written to ./Resources/gene_info.tsv


In [ ]:
#Test gene dict function

for gene_name, gene_info in gene_info_dict.items():
    print(f"Gene: {gene_name}")
    print(f"GC Content (%): {gene_info['GC_content']}")
    print(f"Length (nucleotides): {gene_info['length']}")
    print(f"Nucleotide Sequence: {gene_info['nucleotide_sequence']}")
    print(f"Protein Sequence: {gene_info['protein_sequence']}") 
    print() # spcae inbetween

__THIRD CELL OF PROBLEM 1:__

f. Make a histogram of the GC content of all yeast genes.

In [60]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

gc_contents = [gene_info['GC_content'] for gene_name, gene_info in gene_info_dict.items()]

fig = make_subplots(rows=1, cols=2, subplot_titles=('Standard Histogram', 'Cumulative Histogram'))

fig.add_trace(
    go.Histogram(x=gc_contents, opacity=0.8, histnorm='probability'),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x = gc_contents, opacity = 0.8, cumulative=dict(enabled=True),histnorm='probability'),
    row=1, col=2
)

fig.update_layout(
    title_text='Histograms of GC Content',
    xaxis_title='GC Content (%)',
    yaxis_title='Probability',
    xaxis2_title='GC Content (%)'
)

fig.show()


__FOURTH CELL OF PROBLEM 1:__

g. Make a histogram of the sequence length for all yeast genes.

In [59]:
gc_lengths = [gene_info['length'] for gene_name, gene_info in gene_info_dict.items()]

fig = make_subplots(rows=1, cols=2, subplot_titles=('Standard Histogram', 'Cumulative Histogram'))

fig.add_trace(
    go.Histogram(x=gc_lengths, opacity=0.8, histnorm='probability'),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x = gc_lengths, opacity = 0.8, cumulative=dict(enabled=True),histnorm='probability'),
    row=1, col=2
)

fig.update_layout(
    title_text='Histograms of GC Content',
    xaxis_title='GC length',
    yaxis_title='Probability',
    xaxis2_title='GC length'
)

fig.show()

__FIFTH CELL OF PROBLEM 1:__

h. Make a 2D density plot of GC vs sequence length.

In [61]:
import plotly.express as px
px.density_heatmap(x = gc_contents, y = gc_lengths, marginal_x='rug', marginal_y='histogram')

## 2. PlotLy contour plot of BMI:

From wikipedia:

>The body mass index (BMI) or Quetelet index is a value derived from the mass (weight) and height of an individual. The BMI is defined as the body mass divided by the square of the body height, and is universally expressed in units of kg/m2, resulting from mass in kilograms and height in metres.

Create a function to calculate BMI from height and weight and make a 2D contour chart showing the BMI for heights between 1.4 and 2.2m and weights between 40 and 120kg, indicating the healthy range for BMI.

__FIRST CELL OF PROBLEM 2:__


a. Use numpy's linspace to have 100 points for height and 100 for weight.

b. We are going to use plotly's `Contour` plot, it takes as input a single variable, z. This should be a list of lists, which you can think of as a matrix. For each value of height, you should calculate BMI for all values of weight, put these in a list and the list to a list of lists. 

c. By the end of this cell you need three variables, a list (or numpy array) with the height values, a list (or numpy array) with the weight values, and a list of lists with the bmi values.

In [108]:
import numpy as np
import plotly.graph_objs as go

def bmi_calc(h, w):
    return w/(h **2)

# create 100 points for height and weight
heights = np.linspace(1.4, 2.2, 100)
weights = np.linspace(40, 120, 100)

# find bmi for each combo
bmis = []
for h in heights:
    bmi_row = []
    for w in weights:
        bmi_row.append(bmi_calc(h, w))
    bmis.append(bmi_row)

#print(bmis[1])

__SECOND CELL OF PROBLEM 2:__

d. Use plotly's `Contour` to create a graph like the one below:

![BMI Plot](bmi0.png)

In [155]:
fig = go.Figure(data = go.Contour(
    z=bmis,
    x=weights,  
    y=heights, 
    colorscale='Purples'
))

fig.update_layout(
    xaxis_title="Weight",
    yaxis_title="Height",
    autosize=False,
    width=675,
    height=500
)
fig.show()

__THIRD CELL OF PROBLEM 2:__

e. To be most useful, your contour plot should have only three levels, low bmi, good bmi, and high bmi. The cut-offs for these are 18 and 25, your graph should look like the one below (the colors don't matter).

![BMI Plot 2](bmi.png)

In [160]:

fig = go.Figure(data = go.Contour(
    z=bmis,
    x=weights,  
    y=heights, 
    colorscale='Purples',
    contours=dict(
        type="levels", 
        start=18,
        end=25,
        size=7),
))

fig.update_layout(
    xaxis_title="Weight",
    yaxis_title="Height",
    autosize=False,
    width=675,
    height=500
)
fig.show()